In [1]:
# Imports and magic here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# Globals here

In [2]:
# Read input dataset
input_csv = './train.csv'
df_raw = pd.read_csv(input_csv, index_col = 0)

In [3]:
# Identify predictor variables and response variables, and create corresponding DFs
response = 'Response'
all_predictors = [col for col in df_raw.columns if col not in response]

df_predictors = df_raw[all_predictors]
df_response = df_raw[response]
df_response_with_dummies = pd.get_dummies(df_response, prefix='Response')

In [4]:
# Describe each feature: Number of uniques, number of nulls, type_of_feature, distribution

def get_dummy_features(df, verbose=False, dummy_threshold = 200):
    features_for_dummification = []
    num_rows = len(df)

    for each_feature in df.columns:
        num_uniques = len(df[each_feature].unique())
        num_nulls = df[each_feature].isnull().sum()
        example = df[each_feature].iloc[0]
        
        # Keep track of dummy features
        if isinstance(example, str) or (num_uniques*dummy_threshold<num_rows and isinstance(example, (int, long))): 
            features_for_dummification.append(each_feature)

        if verbose==True:
            print '{}: Uniques: {}/{}. Nulls: {}. Type: {}'.format(each_feature, num_uniques, 
                                                               num_rows, num_nulls, type_of_feature)
            
            
    return features_for_dummification

In [5]:
# Turn chosen features into dummy variables
def create_dummy_features(df_raw, features_for_dummification, verbose=True):
    df_expanded = df_raw

    for each_feature in features_for_dummification:
        if verbose==True:
            print "Expanding variable: {}".format(each_feature)
        df_temp = pd.get_dummies(df_raw[each_feature], prefix=each_feature)
        df_expanded = pd.concat([df_expanded, df_temp], axis = 1, join = 'inner')
        df_expanded.drop(each_feature,inplace=True, axis=1)
        
    return df_expanded

In [6]:
# Since training data and test data have different distributions for "dummyizable" features, it makes sense to
# build out dummy features using a combined distribution
df_oos_sneak_peek = pd.read_csv('./test.csv', index_col = 0)
df_predictors_enhanced = pd.concat([df_predictors, df_oos_sneak_peek], axis = 0, join = 'outer')

In [7]:
df_predictors_enhanced.shape

(79146, 126)

In [8]:
features_for_dummification = get_dummy_features(df_predictors)
features_already_dummy = ['Medical_Keyword_{}'.format(num) for num in range(1,49)]
features_for_dummification = sorted(list(set(features_for_dummification) - set(features_already_dummy)))
print features_for_dummification

['Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 'Family_Hist_1', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Medical_History_11', 'Medical_History_12', 'Medical_History_13', 'Medical_History_14', 'Medical_History_16', 'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 'Medical_History_20', 'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 'Medical_History_26', 'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 'Medical_History_3', 'Medical_History_30', 'Medical_History_31', 'Medical_History_33', 'Medical_History_34', 'Medical_History_35', 'Medical_History_36', 'Medical_History_37', 'Medical_History_38', 'Medical_History_39', 'Medical_History_4', 'Medical_History_40', 'Me

In [9]:
df_expanded_enhanced = create_dummy_features(df_predictors_enhanced, features_for_dummification)
df_expanded = df_expanded_enhanced.drop(df_oos_sneak_peek.index)
df_expanded.shape

Expanding variable: Employment_Info_2
Expanding variable: Employment_Info_3
Expanding variable: Employment_Info_5
Expanding variable: Family_Hist_1
Expanding variable: Insurance_History_1
Expanding variable: Insurance_History_2
Expanding variable: Insurance_History_3
Expanding variable: Insurance_History_4
Expanding variable: Insurance_History_7
Expanding variable: Insurance_History_8
Expanding variable: Insurance_History_9
Expanding variable: InsuredInfo_1
Expanding variable: InsuredInfo_2
Expanding variable: InsuredInfo_3
Expanding variable: InsuredInfo_4
Expanding variable: InsuredInfo_5
Expanding variable: InsuredInfo_6
Expanding variable: InsuredInfo_7
Expanding variable: Medical_History_11
Expanding variable: Medical_History_12
Expanding variable: Medical_History_13
Expanding variable: Medical_History_14
Expanding variable: Medical_History_16
Expanding variable: Medical_History_17
Expanding variable: Medical_History_18
Expanding variable: Medical_History_19
Expanding variable: Me

(59381, 325)

In [ ]:
# # Compute pairwise correlations of all predictors
# df_predictors = df_expanded
# df_predictor_correlations = df_predictors.corr()


In [ ]:
# # Describe pairwise correlations
# _corr_threshold = 0.5

# for pred in all_predictors:
#     df_temp = df_predictor_correlations[pred]
#     max_corr = df_temp[[idx for idx in df_temp.index if pred not in idx]].max()
#     min_corr = df_temp.min()
#     if np.isnan(max_corr)==False:
#         if abs(min_corr)>max_corr:
#             other_pred = df_temp[df_temp==min_corr].index[0]
#             mcorr = min_corr
#         else:
#             other_pred = df_temp[df_temp==max_corr].index[0]
#             mcorr = max_corr
#         if abs(mcorr)>_corr_threshold:
#             print '** ',
#         print 'mcorr({})={:.4f} with {}'.format(pred, mcorr, other_pred)

In [10]:
# Create a test harness: train and test sets
# Using k-fold cross-validation
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

df_predictors_selected = df_expanded.copy(deep=True)
df_response_selected = df_response

# Fill NAs with median estimates (not localized approach yet)
df_predictors_selected.fillna(df_predictors_selected.median(), inplace=True)
df_response_selected.fillna(df_response_selected.median(), inplace=True)

CV_FOLDS = 4
skf = KFold(df_response.index.max(), n_folds=CV_FOLDS)
ctr_fold = 0

# Cross-validation
for idx_train, idx_test in skf:
    print 'Fold #{}\n\t'.format(ctr_fold)
    
    # Build training and test datasets here
    trg_x = df_predictors_selected.ix[idx_train, :].dropna(how='all',axis=0)
    trg_y = df_response_selected.ix[idx_train].dropna(how='all', axis=0)
    tst_x = df_predictors_selected.ix[idx_test, :].dropna(how='all', axis=0)
    tst_y = df_response_selected.ix[idx_test].dropna(how='all', axis=0)
    
    # Insert main model here
#     clf = RandomForestClassifier(n_estimators = 1000) # Score: ~0.226
#     clf = KNeighborsClassifier(n_neighbors = 5, weights = 'distance', algorithm = 'ball_tree', n_jobs = -1)
    clf = GradientBoostingClassifier(n_estimators = 50, verbose=True, random_state=42)
    clf.fit(trg_x, trg_y)

    score = clf.score(tst_x, tst_y)
    
    # Check performance: use accuracy metric
    print 'Score: {}'.format(score)
    
    ctr_fold += 1
    
    
# Final model training/test on ALL training data
clf.fit(df_predictors_selected, df_response_selected)
print 'Final model training score: {}'.format(clf.score(df_predictors_selected, df_response_selected))

Fold #0
	
      Iter       Train Loss   Remaining Time 
         1       83308.0629            6.49m
         2       78730.3698            6.08m
         3       75269.1612            5.86m
         4       72532.5473            5.66m
         5       70283.5824            5.54m
         6       68424.1147            5.41m
         7       66848.2461            5.27m
         8       65495.6481            5.18m
         9       64329.0702            5.05m
        10       63332.1491            4.92m
        20       57786.0080            3.66m
        30       55368.6307            2.43m
        40       54045.7772            1.22m
        50       53133.4262            0.00s
Score: 0.570245150862
Fold #1
	
      Iter       Train Loss   Remaining Time 
         1       83218.5057            6.22m
         2       78645.5464            6.05m
         3       75178.5098            6.27m
         4       72426.1710            6.03m
         5       70231.2163            6.11m
         6 

# Learning curve

The point is to see if we have enough data. We will do this by determining cross-validated training and test scores for different training set sizes.

In [ ]:
# Sample a portion of the dataframe to create a smaller dataframe we can work with for now



In [15]:
from sklearn.cross_validation import train_test_split, StratifiedKFold, ShuffleSplit 
from sklearn.linear_model import LogisticRegression
from sklearn.learning_curve import learning_curve
X = df_predictors_selected
y = df_response_selected

In [18]:
model = LogisticRegression(C=1.0)#.fit(X_train,Y_train)
train_sizes,train_scores,test_scores = learning_curve(model,X,y,train_sizes = np.linspace(0.05,0.1,1),cv = None) 
                                                     
                                         #cv=StratifiedKFold(y,n_folds=5,shuffle=True,random_state=1))

In [20]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
from sklearn import cross_validation
from sklearn.datasets import load_digits
estimator = LogisticRegression()
title = "Learning curve Logistic Regression"
digits = load_digits()
X, y = digits.data, digits.target
cv = cross_validation.ShuffleSplit(digits.data.shape[0], n_iter=100,
                                   test_size=0.2, random_state=0)
plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)
plt.show()

# Model building:

**Make another version of the data where the minority response values are oversampled OR synthetically generated using SMOTE. First use the original dataset and then the altered for each model:[X,y OR X1,y1]**

1. Fit independent models using all features:
    1.1 Logistic Regression
    1.2 Naive Bayes
    1.3 K-Nearest Neighbor



2. Plot learning curve for any (?) model that gives a decent accuracy to see if we have enough data in hand!!



3. Fit ensemble models using all features:
    3.1 RandomForest
    3.2 Gradient Boosting Classifier
    3.3 Voting Classifier (combo of models from Step#1: show that base classifiers are better than a coin toss)
    
 
**Other points to think about:**

1) K-fold cross-validation can be folded into the model: each run will give a score: we can get the median score.


2) Feature selection: We should apply penalty (C which is 1/lambda.. C=0.1 means high penalty. Apply Lasso). Find coefficients and discuss the importance. Look for intuitive explanations. 
2.1) For Logistic Regression and Naive Bayes, we will get probabilities so besides just looking at the median CV score,w ill it be useful to look at the confusion matrix, find TPR, FPR, plot a ROC curve and determine AUC? Is that of interest to Prudential??


3) To simplify the model, if scores aren't awesome, we can even remove features showing multicollinearity by the correlation rule in the book. (Should we be looking at covariance?). Will PCA help in finding out which features are colinear? Because they will be on the same axis.




In [ ]:
### Out of sample testing

In [ ]:
# Read test dataset
df_oos_test = pd.read_csv('./test.csv', index_col = 0)
df_oos_test_expanded = create_dummy_features(df_oos_test, features_for_dummification)

In [ ]:
# Fill NAs
df_oos_test_expanded.fillna(df_oos_test_expanded.median(), inplace=True)

In [ ]:
# Run classifier
df_oos_test_predictions = pd.DataFrame(clf.predict(df_oos_test_expanded), columns = ['Response'], index=df_oos_test_expanded.index)

In [ ]:
# Create submission file

# Timestamp filename
from time import strftime
str_timestamp = strftime('%Y%m%d%H%M%S')

df_oos_test_predictions.to_csv('./submission_{}.csv'.format(str_timestamp))

In [ ]:
### END

In [ ]:
# Account for multi-collinearity
# Identify predictors with marginal value, ensure all pairwise correlations are below a certain threshold.
# Procedure is from "Applied Predictive Modeling" by Max Kuhn, pp. 46-47

_multicollinearity_threshold = 0.60

# # Isolate just the unique pairwise correlations by dropping lower triangle and diagonal elements
df_predictor_correlations = df_predictors.corr()
df_corr_temp = df_predictor_correlations.where((np.triu(np.ones(df_predictor_correlations.shape)) 
                                 - np.identity(len(df_predictor_correlations))).astype(bool))

# Create subset DF with pairwise correlations > threshold
df_temp = df_corr_temp[df_corr_temp.abs()>_multicollinearity_threshold].dropna(how='all',axis=1).dropna(how='all',axis=0)

# Create a list of candidate pairs
candidate_pairs = {}
for pred in df_temp.columns:
    max_abs_corr = df_temp[pred].abs().max()
    other_pred = df_temp[df_temp[pred].abs()==max_abs_corr].index[0]
    candidate_pairs[(pred, other_pred)] = max_abs_corr

# Create a sorted list of candidate pairs
candidate_pairs_sorted = sorted(candidate_pairs, key = lambda pair: candidate_pairs[pair], reverse=True)

# Create a list of predictors that should be dropped because of collinearity
collinear_predictors = []

# Remove predictors one by one
for pair in candidate_pairs_sorted:
    pred_1, pred_2 = pair[0], pair[1]
    avg_corr_1, avg_corr_2 = df_predictor_correlations[pred_1].mean(), df_predictor_correlations[pred_2].mean()
    if avg_corr_1 > avg_corr_2:
        pred_to_remove = pred_1
        other_pred = pred_2
    else:
        pred_to_remove = pred_2
        other_pred = pred_1
    try:
        collinear_predictors.append(pred_to_remove)
        print 'Should remove {}. Has a correlation of {:.2f} with {}.'.format(pred_to_remove, candidate_pairs[pair], other_pred)
    except:
        pass    

In [ ]:
# Detect near-zero variance predictors
near_zero_variance_predictors = []

for predictor in df_predictors.columns:
    pred_mode = df_predictors[predictor].mode()[0]
    pred_2nd_mode = df_predictors[predictor][df_predictors[predictor]!=pred_mode].mode()[0]
    freq_mode = len(df_predictors[predictor][df_predictors[predictor]==pred_mode])
    freq_2nd_mode = len(df_predictors[predictor][df_predictors[predictor]==pred_2nd_mode])
    num_uniques = len(df_predictors[predictor].unique())
    mode_ratio = float(freq_mode)/freq_2nd_mode
    uniques_pct = 100*float(num_uniques)/len(df_predictors)
    if mode_ratio>20 and uniques_pct<0.05:
        print '*** Near-zero variance predictor',
        near_zero_variance_predictors.append(predictor)
    print '{}: Mode ratio={:.2f}, uniques={:.4f}%'.format(predictor, mode_ratio, uniques_pct)

In [ ]:
# Build a subset of predictors by removing collinear and near-zero variance predictors

selected_predictors = list(set(all_predictors) - set(collinear_predictors) - set(near_zero_variance_predictors))
df_predictors_subset = df_predictors.ix[:,selected_predictors]

In [ ]:
# Examine correlations of predictors with response variable
df_selected = df_predictors
important_predictors_for_response = {}

for response_var in df_response.unique():
    response_corrs = df_selected.corrwith(df_response_with_dummies['Response_{}'.format(response_var)]).to_dict()
    sorted_corrs = sorted(response_corrs, key=lambda prd: abs(response_corrs[prd]), reverse=True)
    important_predictors_for_response[response_var] = zip(sorted_corrs, [response_corrs[corr] for corr in sorted_corrs])
